## Imports

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
import seaborn as sns
import matplotlib.pyplot as plt

2024-11-12 09:24:31.466331: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 09:24:31.600497: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 09:24:31.708722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731403471.811587    2221 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731403471.837355    2221 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 09:24:32.105343: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

## Divisão dos dados em X e Y

Label Encoder do y & Normalização do X

In [2]:
dataset_path = 'feature_extraction.csv' 
data = pd.read_csv(dataset_path)

X = data.drop(columns=['file_id', 'class', 'fold']) 
y = data['class'] 

#One Hot Encoder de y
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

#Normalização de X
scaler = MinMaxScaler() 
X = scaler.fit_transform(X)

print(pd.DataFrame(X).head())
print(np.unique(y))

         0         1         2         3         4         5         6   \
0  0.683183  0.017093  0.621973  0.469777  0.383277  0.638431  0.505458   
1  0.718800  0.105597  0.460109  0.637143  0.377176  0.563590  0.472180   
2  0.728638  0.092748  0.416206  0.685898  0.283718  0.519607  0.469577   
3  0.478898  0.036833  0.516766  0.653926  0.260736  0.470929  0.489646   
4  0.496398  0.700244  0.422515  0.824047  0.313228  0.513074  0.495865   

         7         8         9   ...        13        14        15        16  \
0  0.035471  0.092136  0.036746  ...  0.077481  0.165125  0.026641  0.279333   
1  0.138060  0.186574  0.273884  ...  0.395330  0.385121  0.297618  0.668237   
2  0.104300  0.098335  0.046947  ...  0.325160  0.241995  0.111223  0.464770   
3  0.139243  0.087479  0.104135  ...  0.125955  0.218403  0.084630  0.418222   
4  0.135814  0.168407  0.233478  ...  0.402947  0.331131  0.254352  0.459972   

         17        18        19        20        21   22  
0  0.0266

## Número de camadas escondidas

Duas camadas escondidas já são capazes de representar qualquer relação entre os dados, mesmo aquelas que não podem ser representadas por equações. Mais do que duas camadas escondidas só são necessárias em problemas ainda mais complexos como séries temporais e visão computacional, onde há uma certa inter-relação entre as dimensões que os dados contêm

In [3]:
n_camadas = [2, 3]
print ("Range do nº de camadas: ", n_camadas)

Range do nº de camadas:  [2, 3]


## Número de neurónios em cada camada escondida

Abordagens:  
O número de neurônios escondidos deve estar entre o tamanho da camada de entrada e o da camada de saída.  
O número de neurônios escondidos deve ser 2/3 do tamanho da camada de entrada, mais o tamanho da camada de saída

In [4]:
n_neur = [0] * 3
input_dim = X.shape[1]
output_dim = len(label_encoder.classes_)
#Neurónios na camada escondida 1
n_neur [0] = int((input_dim + output_dim)/2)
#Neurónios na camada escondida 2
n_neur [1] = int((input_dim + output_dim)/3)
#Neurónios na camada escondida 3
n_neur [2] = int((input_dim + output_dim)/4)
print ("Número de neurónios escondidos: ", n_neur)

Número de neurónios escondidos:  [16, 11, 8]


## Inicialização dos pesos

Função gaussiana para inicializar os pesos

In [5]:
def initialize_weights(shape, dtype=None):
    mean = 0.0
    stddev = 0.01
    return tf.constant(np.random.normal(loc=mean, scale=stddev, size=shape), dtype=dtype)

## Ajuste do Learning Rate ao longo da aprendizagem

Ajustar o Learning Rate para metade do seu valor de 10 em 10 epochs

In [6]:
def scheduler(epoch, lr): 
    if epoch > 0 and epoch % 10 == 0: 
        return lr / 2 
    return lr
lr_scheduler = LearningRateScheduler(scheduler)

## Early Stop

Verificar se a validation loss (no fold de desenvolvimento) não melhora em 10 epochs consecutivos, e para o treino caso isso aconteça

In [7]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

## Grid Search

Função que faz um grid search para encontrar o melhor learning rate para o otimizador Adam e o melhor número de epochs e batch size para o modelo. Os valores em cada grid foram pedidos ao ChatGPT. A cada modelo é aplicada uma 10-cross validation, com 8 folds a serem usados para treino, 1 para teste e 1 para desenvolvimento

In [29]:
def grid_search(model):
    learning_rate = [0.1, 0.01, 0.001]
    epochs = [50, 100, 200]
    batch_size = [16, 32, 64, 128]
    best_model = None
    best_model_accuracy = 0
    for i in learning_rate:
        for j in epochs:
            for m in batch_size:
                adam_optimizer = tf.keras.optimizers.Adam(learning_rate=i)
                # Definir o modelo
                model = model
            
                kf = KFold(n_splits=10, shuffle=True, random_state=42)
                folds = list(kf.split(X))
                all_accuracies = []
                all_confusion_matrices = []
                all_losses = []
                for fold_idx in range(len(folds)):
                    train_val_indices, test_indices = folds[fold_idx]

                    dev_fold_idx = (fold_idx + 1) % len(folds)
                    train_indices, dev_indices = folds[dev_fold_idx]

                    X_train, y_train = X[train_val_indices], y[train_val_indices]
                    X_dev, y_dev = X[dev_indices], y[dev_indices]

                    X_test, y_test = X[test_indices], y[test_indices]

                    model.compile(optimizer=adam_optimizer, loss='sparse_categorical_crossentropy', metrics=["accuracy"])

                    model.fit(X_train, y_train, epochs=j, batch_size=m, validation_data=(X_dev, y_dev), callbacks=[lr_scheduler, early_stopping])

                    loss, accuracy = model.evaluate(X_test, y_test)

                    all_accuracies.append(accuracy) 
                    y_pred = model.predict(X_test) 
                    y_pred_classes = np.argmax(y_pred, axis=1) 
                    cm = confusion_matrix(y_test, y_pred_classes) 
                    all_confusion_matrices.append(cm)
                    all_losses.append(loss)

                accumulated_cm = np.sum(all_confusion_matrices, axis=0) 
                accumulated_cm_percentage = accumulated_cm / np.sum(accumulated_cm) * 100
                mean_accuracy = np.mean(all_accuracies) 
                std_accuracy = np.std(all_accuracies)
                mean_loss = np.mean(all_losses)
                
                if best_model == None or mean_accuracy > best_model_accuracy:
                    best_model = model
                    best_model_accuracy = mean_accuracy
                    best_model_cfm = accumulated_cm_percentage
                    best_model_loss = mean_loss
                    best_model_std = std_accuracy
                    best_model_lr = i
                    best_model_epochs = j
                    best_model_batch_size = m
    return best_model, best_model_accuracy, best_model_cfm, best_model_loss, best_model_std, best_model_lr, best_model_epochs, best_model_batch_size

## Escolha de funções de ativação

Para as camadas escondidas testámos modelos com funções de ativação ReLU e funções de ativação LeakyReLU  
"The ReLU function is the default activation function for hidden layers in modern MLP and CNN neural network models"  
"(The LeakyReLU function) does not have any linear component with zero derivatives (slopes). Therefore, it can avoid the dying ReLU problem"  
"The same usage of the ReLU function is also valid for the leaky ReLU function"  

Para as camadas de output usámos sempre a função de ativação softmax  
"We must use the softmax function in the output layer of a multiclass classification problem"

In [30]:
models = []

## MLP com 2 camadas escondidas e funções de ativação ReLU

In [31]:
model = tf.keras.models.Sequential([ 
    tf.keras.layers.Input(shape=(X.shape[1],)), 
    tf.keras.layers.Dense(n_neur[0], kernel_initializer=initialize_weights, activation='relu'), 
    tf.keras.layers.Dense(n_neur[1], kernel_initializer=initialize_weights, activation='relu'), 
    tf.keras.layers.Dense(output_dim, activation='softmax', kernel_initializer=initialize_weights) 
])
models.append(model)

## MLP com 2 camadas escondidas e funções de ativação LeakyReLU

In [32]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(input_dim,)),
    tf.keras.layers.Dense(n_neur[0], kernel_initializer=initialize_weights),
    tf.keras.layers.LeakyReLU(negative_slope=0.01),
    tf.keras.layers.Dense(n_neur[1], kernel_initializer=initialize_weights),
    tf.keras.layers.LeakyReLU(negative_slope=0.01),
    tf.keras.layers.Dense(output_dim, activation='softmax', kernel_initializer=initialize_weights)
])
models.append(model)

## MLP com 3 camadas escondidas e funções de ativação ReLU

In [33]:
model = tf.keras.models.Sequential([ 
    tf.keras.layers.Input(shape=(X.shape[1],)), 
    tf.keras.layers.Dense(n_neur[0], kernel_initializer=initialize_weights, activation='relu'), 
    tf.keras.layers.Dense(n_neur[1], kernel_initializer=initialize_weights, activation='relu'),
    tf.keras.layers.Dense(n_neur[2], kernel_initializer=initialize_weights, activation='relu'),
    tf.keras.layers.Dense(output_dim, activation='softmax', kernel_initializer=initialize_weights) 
])
models.append(model)

## MLP com 3 camadas escondidas e funções de ativação LeakyReLU

In [34]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(input_dim,)),
    tf.keras.layers.Dense(n_neur[0], kernel_initializer=initialize_weights),
    tf.keras.layers.LeakyReLU(negative_slope=0.01),
    tf.keras.layers.Dense(n_neur[1], kernel_initializer=initialize_weights),
    tf.keras.layers.LeakyReLU(negative_slope=0.01),
    tf.keras.layers.Dense(n_neur[2], kernel_initializer=initialize_weights),
    tf.keras.layers.LeakyReLU(negative_slope=0.01),
    tf.keras.layers.Dense(output_dim, activation='softmax', kernel_initializer=initialize_weights)
])
models.append(model)

## Finding the best MLP model

In [36]:
best_model = None
best_model_accuracy = 0
for model in models:
    model, model_accuracy, model_cfm, model_loss, model_std, model_lr, model_epochs, model_batch_size = grid_search(model)
    if best_model == None or model_accuracy > best_model_accuracy:
        best_model = model
        best_model_accuracy = model_accuracy
        best_model_cfm = model_cfm
        best_model_loss = model_loss
        best_model_std = model_std
        best_model_lr = model_lr
        best_model_epochs = model_epochs
        best_model_batch_size = model_batch_size

Epoch 1/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1074 - loss: 2.2788 - val_accuracy: 0.1053 - val_loss: 2.2725 - learning_rate: 0.1000
Epoch 2/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1108 - loss: 2.2831 - val_accuracy: 0.1247 - val_loss: 2.2689 - learning_rate: 0.1000
Epoch 3/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1058 - loss: 2.2718 - val_accuracy: 0.1053 - val_loss: 2.2820 - learning_rate: 0.1000
Epoch 4/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1088 - loss: 2.2727 - val_accuracy: 0.1201 - val_loss: 2.2806 - learning_rate: 0.1000
Epoch 5/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 0s 967us/step - accuracy: 0.1190 - loss: 2.2744 - val_accuracy: 0.1201 - val_loss: 2.2740 - learning_rate: 0.1000
Epoch 6/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1166 - loss: 2.2812 - val_accuracy: 0.1201 - val_loss: 2.2713 - learning_rate: 0.1000
Epoch 7/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1145 - loss: 2.2734 

492/492 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1196 - loss: 2.2794 - val_accuracy: 0.1157 - val_loss: 2.2896 - learning_rate: 0.1000
Epoch 2/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0989 - loss: 2.2770 - val_accuracy: 0.1168 - val_loss: 2.2662 - learning_rate: 0.1000
Epoch 3/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1128 - loss: 2.2771 - val_accuracy: 0.1111 - val_loss: 2.2833 - learning_rate: 0.1000
Epoch 4/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 0s 923us/step - accuracy: 0.1187 - loss: 2.2737 - val_accuracy: 0.1111 - val_loss: 2.2826 - learning_rate: 0.1000
Epoch 5/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step - accuracy: 0.1146 - loss: 2.2725 - val_accuracy: 0.1180 - val_loss: 2.2636 - learning_rate: 0.1000
Epoch 6/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1025 - loss: 2.2758 - val_accuracy: 0.1180 - val_loss: 2.2734 - learning_rate: 0.1000
Epoch 7/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1088 - loss: 2.2799 - val_acc

492/492 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1137 - loss: 2.2783 - val_accuracy: 0.1168 - val_loss: 2.2725 - learning_rate: 0.1000
Epoch 2/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1138 - loss: 2.2782 - val_accuracy: 0.1065 - val_loss: 2.2803 - learning_rate: 0.1000
Epoch 3/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1084 - loss: 2.2790 - val_accuracy: 0.1168 - val_loss: 2.2822 - learning_rate: 0.1000
Epoch 4/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1208 - loss: 2.2803 - val_accuracy: 0.1031 - val_loss: 2.2672 - learning_rate: 0.1000
Epoch 5/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 1s 953us/step - accuracy: 0.1038 - loss: 2.2789 - val_accuracy: 0.1065 - val_loss: 2.2790 - learning_rate: 0.1000
Epoch 6/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1084 - loss: 2.2803 - val_accuracy: 0.1168 - val_loss: 2.2711 - learning_rate: 0.1000
Epoch 7/50
492/492 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1184 - loss: 2.2792 - val_accur

246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1160 - loss: 2.2718 - val_accuracy: 0.1111 - val_loss: 2.2926 - learning_rate: 0.1000
Epoch 2/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1087 - loss: 2.2720 - val_accuracy: 0.1180 - val_loss: 2.2549 - learning_rate: 0.1000
Epoch 3/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1182 - loss: 2.2721 - val_accuracy: 0.1226 - val_loss: 2.2691 - learning_rate: 0.1000
Epoch 4/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1203 - loss: 2.2711 - val_accuracy: 0.1111 - val_loss: 2.2689 - learning_rate: 0.1000
Epoch 5/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1146 - loss: 2.2771 - val_accuracy: 0.1157 - val_loss: 2.2648 - learning_rate: 0.1000
Epoch 6/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1195 - loss: 2.2723 - val_accuracy: 0.1180 - val_loss: 2.2668 - learning_rate: 0.1000
Epoch 7/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1153 - loss: 2.2675 - val_accurac

246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1093 - loss: 2.2773 - val_accuracy: 0.1134 - val_loss: 2.2635 - learning_rate: 0.1000
Epoch 2/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1194 - loss: 2.2678 - val_accuracy: 0.0997 - val_loss: 2.2710 - learning_rate: 0.1000
Epoch 3/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1066 - loss: 2.2672 - val_accuracy: 0.1249 - val_loss: 2.2599 - learning_rate: 0.1000
Epoch 4/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1159 - loss: 2.2728 - val_accuracy: 0.1168 - val_loss: 2.2703 - learning_rate: 0.1000
Epoch 5/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1092 - loss: 2.2754 - val_accuracy: 0.1191 - val_loss: 2.2628 - learning_rate: 0.1000
Epoch 6/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1172 - loss: 2.2691 - val_accuracy: 0.0997 - val_loss: 2.2610 - learning_rate: 0.1000
Epoch 7/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 989us/step - accuracy: 0.1139 - loss: 2.2669 - val_accur

...

Epoch 9/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1102 - loss: 2.2583 - val_accuracy: 0.1247 - val_loss: 2.2618 - learning_rate: 0.0100
Epoch 10/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1169 - loss: 2.2619 - val_accuracy: 0.1098 - val_loss: 2.2627 - learning_rate: 0.0100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 835us/step - accuracy: 0.1027 - loss: 2.2594   
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 1/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1133 - loss: 2.2552 - val_accuracy: 0.1306 - val_loss: 2.2489 - learning_rate: 0.0100
Epoch 2/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1192 - loss: 2.2602 - val_accuracy: 0.1168 - val_loss: 2.2481 - learning_rate: 0.0100
Epoch 3/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1137 - loss: 2.2576 - val_accuracy: 0.1168 - val_loss: 2.2496 - learning_rate: 0.0100
Epoch 4/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1134 - loss: 2.2625 - val_accuracy: 0.1180 - val_l

Epoch 9/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1135 - loss: 2.2574 - val_accuracy: 0.1111 - val_loss: 2.2629 - learning_rate: 0.0100
Epoch 10/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1112 - loss: 2.2593 - val_accuracy: 0.1111 - val_loss: 2.2623 - learning_rate: 0.0100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1077 - loss: 2.2729 
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Epoch 1/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.1169 - loss: 2.2573 - val_accuracy: 0.1134 - val_loss: 2.2621 - learning_rate: 0.0100
Epoch 2/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1077 - loss: 2.2564 - val_accuracy: 0.1134 - val_loss: 2.2629 - learning_rate: 0.0100
Epoch 3/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1062 - loss: 2.2547 - val_accuracy: 0.1134 - val_loss: 2.2624 - learning_rate: 0.0100
Epoch 4/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1080 - loss: 2.2605 - val_accuracy: 0.1054 - val_loss:

Epoch 9/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1184 - loss: 2.2580 - val_accuracy: 0.1317 - val_loss: 2.2615 - learning_rate: 0.0100
Epoch 10/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1081 - loss: 2.2577 - val_accuracy: 0.1214 - val_loss: 2.2611 - learning_rate: 0.0100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 903us/step - accuracy: 0.1164 - loss: 2.2676
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Epoch 1/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.1155 - loss: 2.2602 - val_accuracy: 0.1053 - val_loss: 2.2562 - learning_rate: 0.0100
Epoch 2/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1171 - loss: 2.2562 - val_accuracy: 0.1156 - val_loss: 2.2572 - learning_rate: 0.0100
Epoch 3/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1156 - loss: 2.2592 - val_accuracy: 0.1259 - val_loss: 2.2561 - learning_rate: 0.0100
Epoch 4/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1001 - loss: 2.2628 - val_accuracy: 0.1236 - val_loss

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1162 - loss: 2.2578 - val_accuracy: 0.1180 - val_loss: 2.2525 - learning_rate: 0.0100
Epoch 10/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1133 - loss: 2.2596 - val_accuracy: 0.1157 - val_loss: 2.2536 - learning_rate: 0.0100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 628us/step - accuracy: 0.0972 - loss: 2.2626
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Epoch 1/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.1106 - loss: 2.2599 - val_accuracy: 0.1168 - val_loss: 2.2682 - learning_rate: 0.0100
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1154 - loss: 2.2637 - val_accuracy: 0.1214 - val_loss: 2.2676 - learning_rate: 0.0100
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1097 - loss: 2.2641 - val_accuracy: 0.1260 - val_loss: 2.2675 - learning_rate: 0.0100
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1099 - loss: 2.2604 - val_accuracy: 0.0997 - val_loss: 2.2689 - learning_rate

SystemError: /opt/conda/conda-bld/python-split_1661469458365/work/Objects/listobject.c:318: bad argument to internal function

In [ ]:
print(best_model.summary())
print("Learning rate: ", best_model_lr)
print("Number of epochs: ", best_model_epochs)
print("Batch Size: ", best_model_batch_size)
print("Accuracy: ", best_model_accuracy)
print("Loss: ", best_model_loss)
print("Standadr Deviation: ", best_model_std)

cm = pd.DataFrame(best_model_cfm, index=label_encoder.classes_, columns=label_encoder.classes_)

plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues')

plt.title('Matriz de Confusão')
plt.xlabel('Previsão')
plt.ylabel('Verdadeiro')
plt.show()

## Referências

https://towardsdatascience.com/how-to-choose-the-right-activation-function-for-neural-networks-3941ff0e6f9c  
https://medium.com/thedeephub/learning-rate-and-its-strategies-in-neural-network-training-270a91ea0e5c  
https://www.kaggle.com/code/androbomb/simple-nn-with-python-multi-layer-perceptron  
https://medium.com/ensina-ai/rede-neural-perceptron-multicamadas-f9de8471f1a9  
https://iaexpert.academy/2020/05/04/quantas-camadas-escondidas-e-quantos-neuronios-incluir-numa-rede-neural-artificial/
https://towardsdatascience.com/how-to-choose-the-right-activation-function-for-neural-networks-3941ff0e6f9c  
https://github.com/enochkan/building-from-scratch/blob/main/optimizers/adam-optimizer-from-scratch.ipynb  
https://www.geeksforgeeks.org/ml-mini-batch-gradient-descent-with-python/